# <center> HomeworkSF:  проверка данных на нормальность и на наличие корреляции

In [40]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_ind
from scipy.stats import pearsonr
import numpy as np

Информация по мидиям Петербурга и Магадана, представлена в виде двух массивов petersburg и magadan:

In [41]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

1. Объединяем два массива petersburg и magadan в один DataFrame

In [42]:
df_p = pd.DataFrame(petersburg, columns=['petersburg'])
#display(df_p)
df_m = pd.DataFrame(magadan, columns=['magadan'])
#display(df_m)
data = pd.concat([df_p, df_m], axis=1)
display(data)

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


2. Проверка данных на нормальность. Применим тест Шапиро-Уилка

In [43]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

alpha = 0.05 # уровень значимости

_, p = shapiro(data[['petersburg']])
print('Для выборки "petersburg": p=%.3f' % p)

if p > alpha:
	print(H0)
else:
	print(Ha)

_, p = shapiro(data[['magadan']])
print('\nДля выборки "magadan": p=%.3f' % p)

if p > alpha:
	print(H0)
else:
	print(Ha)

Для выборки "petersburg": p=1.000
Данные распределены нормально

Для выборки "magadan": p=0.036
Данные не распределены нормально (мы отвергаем H0)


3. Проводим тест на корреляцию

In [44]:
data.corr()

,petersburg,magadan
petersburg,1.000000,0.255095
magadan,0.255095,1.000000


Коэффициент корреляции равен 0.25, что говорит нам об отсутствии связи или очень слабой связи между признаками. Для теста на корреляцию выбираем корреляцию Пирсона т.к. признаки являются непрерывными и один из них нормально распределен.

In [45]:
H0 = 'Нет значимой разницы между средним размером раковины мидий из Петербурга и Магадана'
Ha = 'Есть значимая разница между средним размером раковины мидий из Петербурга и Магадана'

corr, p = pearsonr(petersburg[0:6], magadan[0:6])
corr, p = pearsonr(petersburg[0:6], magadan[0:6])
print(corr,p)

'''from scipy.stats import spearmanr
corr, p = spearmanr(petersburg[0:6], magadan[0:6])
print(corr,p)

from scipy.stats import f_oneway
corr, p = f_oneway(petersburg, magadan)'''

if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными: ", H0)
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными: {Ha}")

0.2605889468556447 0.6179644343046113
0.6179644343046113 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными:  Нет значимой разницы между средним размером раковины мидий из Петербурга и Магадана


Вывод. Согласно поученным результатам теста на корреляцию, мы можем сказать, что данные из Петербурга и Магадана не коррелируют между собой/связь отсутствует. Этот же вывод мы можем сделать на основании полученного значения коэффициента корреляции Пирсона (corr = 0.25 - слабая связь).

4. Проведём тест на сравнение выборок и определим разницу в среднем размере мидии в зависимости от города-производителя

In [46]:
H0 = 'Нет значимой разницы между средним размером раковины мидий, выращенных в Петербурге и в Магадане'
Ha = 'Есть значимая разница между средним размером раковины мидий, выращенных в Петербурге и в Магадане'

Зависимая переменная (средний размер раковины мидии) является количественной. Группы происходят из разных совокупностей. Будем использовать независимый T-тест.

In [47]:
def t_test(df):    
    l=df.shape[0]    
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['petersburg'][0:l-1], df['magadan'][0:l-1], equal_var=True)
    print(test_results)
    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу: {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу: {Ha}")

t_test(data)


*** Результаты независимого T-теста ***
Ttest_indResult(statistic=3.0350285894495013, pvalue=0.010369233123633553)
0.01 <= 0.05. Мы отвергаем нулевую гипотезу: Есть значимая разница между средним размером раковины мидий, выращенных в Петербурге и в Магадане


In [48]:
# print(data['petersburg'].mean(), data['magadan'].mean())
r = np.abs((data['petersburg'].mean()-data['magadan'].mean()))
print("Разница между средним размером раковины мидий из Петербурга и Магадана составляет:", round(r,3))

Разница между средним размером раковины мидий из Петербурга и Магадана составляет: 0.025


Вывод. Согласно Т-тесту на сравнение выборок, наблюдается значимая разница между средним размером раковины мидий из разных городов-производителей, которая составляет 0.025. 